In [1]:
import pandas as pd
import numpy as np

import folium
from datetime import datetime,date
from arcgis.geocoding import geocode
from arcgis.gis import GIS
from arcgis.geometry import Point
from folium.plugins import HeatMap
from ipywidgets import interact, Dropdown
from ipywidgets import widgets
from folium.plugins import MarkerCluster
from geopy.geocoders import Nominatim
from ipywidgets import interactive
from IPython.display import display, clear_output
import seaborn as sns
import spacial_temporal_permutation_test
from pykrige.ok import OrdinaryKriging
from sklearn.metrics.pairwise import pairwise_distances

In [3]:
t_x =pd.read_csv('/home/qb/ML_Group_Assignment/rain_base.csv',parse_dates=['start', 'end'])
t_x.head()

,Unnamed: 0,station,latitude,longitude,distance,event,year,title,volume,start,end,location
0,301,samut prakan,8.426269,99.445290,364.062139,Rain,2017,Raining at The Lower Southern Thailand,0.041667,2017-01-02 06:38:22,2017-01-02 07:38:22,samut prakan
1,395,samut prakan,9.780360,99.078864,281.928179,Rain,2017,Raining at National Highway 41,0.041667,2017-01-02 14:48:58,2017-01-02 15:48:58,samut prakan
2,574,samut prakan,8.427628,99.837365,359.196800,Rain,2017,Raining at The Lower Southern Thailand,0.041667,2017-01-03 05:39:39,2017-01-03 06:39:39,samut prakan
3,645,samut prakan,13.859107,100.484139,19.391068,Rain,2017,Raining at Nonthaburi,0.041667,2017-01-03 13:13:21,2017-01-03 14:13:21,samut prakan
4,1412,samut prakan,9.783959,99.077680,281.726952,Rain,2017,Raining at National Highway 41,0.041667,2017-01-05 19:35:05,2017-01-05 20:35:05,samut prakan


In [10]:
t=t_x[t_x['distance'] < 1]
locations = t_x['location'].unique().tolist()

In [5]:
stations_lat_long = {'latitude':[], 'longitude':[], 'location':[]}
gis = GIS()
def get_lat_long_by_state(location):
    if location  != None or location != '' :
        geolocator = Nominatim(user_agent="myGeopyApp/1.0")
        esrihq = geocode(location)
        if esrihq != None:
            latitude, longitude = esrihq[0]['location']['x'], esrihq[0]['location']['y']
            return latitude, longitude
for loc in locations:
    longitude, latitude = get_lat_long_by_state(loc)
    stations_lat_long['latitude'].append(latitude)
    stations_lat_long['longitude'].append(longitude)
    stations_lat_long['location'].append(loc)

In [7]:
stations_lat_long =pd.DataFrame(stations_lat_long)
stations_lat_long

,latitude,longitude,location
0,13.599468,100.596887,samut prakan
1,13.731625,100.536658,chulalongkorn-hospital
2,13.547513,100.273680,samut sakhon
3,47.715870,-111.686373,thonburi-power


In [11]:
# my_map = folium.Map(location=map_center, zoom_start=12)
map_center = [13.599468 ,100.596887]  
m = folium.Map(location=map_center, zoom_start=12)
marker_cluster = MarkerCluster().add_to(m)

for i,k in enumerate(range(0,stations_lat_long.shape[0])):
    folium.Marker(
        location=[stations_lat_long['latitude'].loc[i], stations_lat_long['longitude'].loc[i]],
        popup=f"Value: {stations_lat_long['location'].loc[i]}",
        icon=folium.Icon(color="blue")
    ).add_to(m)


def update_map_button(b, marker_cluster, m):
    # Remove the current marker cluster and add a new one
#     for marker in list(marker_cluster._children.values()):
#         marker_cluster._children.pop(marker)
    # Filter the data based on selected year and location
    marker_cluster = MarkerCluster().add_to(m)
    year = year_selector.value
    location = location_dropdown.value
    p = t[(t['station'] == location) & (t['year'] == year)]
    
    marker_cluster = MarkerCluster().add_to(m)

    # Add circles to the map based on the filtered data
    for w in range(p.shape[0]):
        latitude = p.iloc[w]['latitude']
        longitude = p.iloc[w]['longitude']
        distance = p.iloc[w]['distance']
        
        # Add a circle for each data point to the marker cluster
        folium.
        folium.Circle(
            location=(latitude, longitude),
            radius=(distance+1)**2,  # Circle radius (with a small offset)
            color="red",  # Circle border color
            fill=True,  # Fill the circle
            fill_color="red",  # Circle fill color
            fill_opacity=0.5  # Fill opacity (0-1)
        ).add_to(marker_cluster)
    
    # Display the updated map
    clear_output(wait=True)
    display(location_dropdown, year_selector, update_button)
    display(m)
    # Return the updated map (to display it interactively)
    return m


In [12]:
# Create the dropdown widget to select the year

#visualise traffic data

# Function to update the map based on selected year and location
year_selector = Dropdown(
    options=t['year'].unique(),
    value=2020,
    description="Year:"
)
location_dropdown = Dropdown(
    options=stations_lat_long['location'].unique(),
    value='samut prakan',  # default value
    description='Location:',
    disabled=False,
)



update_button = widgets.Button(description="Update Map")
# update_button.on_click(update_map_button,)
update_button.on_click(lambda b: update_map_button(b, marker_cluster, m))
display(year_selector, location_dropdown, update_button)
# Use interact to dynamically update the map
display(m)

Dropdown(description='Location:', index=1, options=('samut prakan', 'chulalongkorn-hospital', 'samut sakhon', …

Dropdown(description='Year:', index=2, options=(2017, 2018, 2019, 2020, 2021, 2022, 2023), value=2019)

Button(description='Update Map', style=ButtonStyle())